# Feldspar Solid Solution - Berman Compatible

Required Python code to load the phase library.

In [ ]:
from thermoengine import core, phases, model

### Listing of solution phases compatible with Berman inherited from the MELTS model
get_phase_info returns an ordered list of *pure* and *solution* phases

In [ ]:
phase_info,info_files = phases.get_phase_info()
phase_info['solution']

### Get access to a thermodynamic database (by default, the Berman (1988)/MELTS database).

In [ ]:
modelDB = model.Database()

### Create a Python reference to the feldspar solution phase class.
The required phase abbreviation method argument is listed above.

In [ ]:
Feldspar = modelDB.get_phase('Fsp')

### Obtain some properties of the selected feldspar solution  
Name, formulas of endmembers, molecular weights of endmembers, abbreviation, number of endmember components, names of endmembers

In [ ]:
print (Feldspar.props['phase_name'])
print (Feldspar.props['formula'])
print (Feldspar.props['molwt'])
print (Feldspar.props['abbrev'])
print (Feldspar.props['endmember_num'])
print (Feldspar.props['endmember_name'])

## Specify the composition of the solution
#### Composition input in wt% oxides (treated as grams of oxides)
Sandine composition in wt% oxides, taken from Deer, Howie and Zussman, Table 30, entry 9: Sanidine from a rhyolite in Mitchell Mesa, Texas.

In [ ]:
mol_oxides = core.chem.format_mol_oxide_comp({'SiO2':67.27,'Al2O3':18.35, 'FeO':0.92, 'CaO':0.15, 
                                              'Na2O':6.45, 'K2O':7.05, 'H2O':0.16}, convert_grams_to_moles=True)

### Convert analytical composition to moles of endmember components
Note that a method is called - *test_endmember_comp()* - to test the validity of the projected composition
#### (1st method) A default projection method based on generic least sqaures ...

In [ ]:
moles_end,oxide_res = Feldspar.calc_endmember_comp(mol_oxide_comp=mol_oxides, output_residual=True)
for i in range(0,Feldspar.props['endmember_num']):
    print ("mole number of {0:10.10s} = {1:13.6e}".format(Feldspar.props['endmember_name'][i], moles_end[i]))
if not Feldspar.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")

#### (2nd method) An intrinsic projection method tailored specifically to the solution phase of interest ...

In [ ]:
moles_end,oxide_res = Feldspar.calc_endmember_comp(mol_oxide_comp=mol_oxides, method='intrinsic', output_residual=True)
for i in range(0,Feldspar.props['endmember_num']):
    print ("mole number of {0:10.10s} = {1:13.6e}".format(Feldspar.props['endmember_name'][i], moles_end[i]))
if not Feldspar.test_endmember_comp(moles_end):
    print ("Calculated composition is infeasible!")

### Some convenience methods to manipulate composition information
➜ Moles of endmember components => Molar sum  
➜ Moles of endmember components => Moles of elements (standard order)  
➜ Moles of endmember components => Mole fractions of endmember components  
➜ Moles of elements (standard order) => Moles of endmember components of the phase  
➜ Moles of elements (standard order) => Total moles of endmember components of the phase  
➜ Moles of elements (standard order) => Total mass of the phase (g)

In [ ]:
print ('Total moles of endmembers: ', Feldspar.convert_endmember_comp(moles_end,output='total_moles'))
mol_elm = Feldspar.convert_endmember_comp(moles_end,output='moles_elements')
print ('Mole fractions of endmembers: ', Feldspar.convert_endmember_comp(moles_end,output='mole_fraction'))
print ('Moles of endmembers: ', Feldspar.convert_elements(mol_elm, output='moles_end'))
print ('Total moles of endmembers: ', Feldspar.convert_elements(mol_elm, output='total_moles'))
print ('Total grams of phase: ', Feldspar.convert_elements(mol_elm, output='total_grams'))

## Compute activities and chemical potentials of endmember components:
➜ Moles of components, T (K), P (bars) => activities of endmember components  
➜ Moles of components, T (K), P (bars) => chemical potentials of endmember components (J)

In [ ]:
t = 1000
p = 1000

In [ ]:
names = Feldspar.props['endmember_name']
forms = Feldspar.props['formula']
for i in range(0,Feldspar.props['endmember_num']):
    print ("chemical potential of {0:15.15s} ({1:15.15s}) = {2:15.2f} J, activity = {3:10.6f}".format(
        names[i], forms[i], Feldspar.chem_potential(t, p, mol=moles_end, endmember=i),
        Feldspar.activity(t, p, mol=moles_end, endmember=i)))

## Molar derivatives of activities:
➜ Moles of components, T (K), P (bars) => d(activities of endmember components)/d(Moles of components)

In [ ]:
for i in range(0,Feldspar.props['endmember_num']):
    dadm = Feldspar.activity(t,p,mol=moles_end,deriv={'dmol':1},endmember=i)[0]
    print ("Derivative of a[{0:s}] with respect to:".format(names[i]))
    for j in range(0,Feldspar.props['endmember_num']):
        print ("    {0:15.15s} = {1:13.6e}".format(names[j], dadm[j]))

## Gibbs free energy and its compositional derivatives:
➜ Moles of components, T (K), P (bars) => Gibbs free energy (J)  
➜ Moles of components, T (K), P (bars) => d(Gibbs free energy)/d(Moles of components) (J)  
➜ Moles of components, T (K), P (bars) => d^2(Gibbs free energy)/d(Moles of components)^2 (J)  
➜ Moles of components, T (K), P (bars) => d^3(Gibbs free energy)/d(Moles of components)^3 (J)

In [ ]:
print ('Gibbs free energy = {0:12.2f} J'.format(Feldspar.gibbs_energy(t, p, mol=moles_end)))
print ("")
dgdm = Feldspar.gibbs_energy(t, p, mol=moles_end, deriv={'dmol':1})[0]
d2gdm2 = Feldspar.gibbs_energy(t, p, mol=moles_end, deriv={'dmol':2})[0]
for i in range (0, Feldspar.props['endmember_num']):
    print ('dg/dm[{0:>2d}] = {1:13.6e}     d2gdm2[{0:>2d}][*]:  '.format(i, dgdm[i], i), end=' ')
    for j in range (0, Feldspar.props['endmember_num']):
        print ('{0:13.6e}'.format(d2gdm2[i, j]), end=' ')
    print()
print ("")
d3gdm3 = Feldspar.gibbs_energy(t, p, mol=moles_end, deriv={'dmol':3})[0]
for i in range (0, Feldspar.props['endmember_num']):
    print('d3gdm3[{0:>2d}][*][*]: '.format(i), end=' ')
    for j in range (0, Feldspar.props['endmember_num']):
        for k in range (0, Feldspar.props['endmember_num']):
            print ('{0:13.6e}'.format(d3gdm3[i, j, k]), end=' ')
        print ('  ', end=' ')
    print ()

## Enthalpy, Entropy, and molar derivatives:
➜ Moles of components, T (K), P (bars) => enthalpy (J)  
➜ Moles of components, T (K), P (bars) => entropy (J/K)  
➜ Moles of components, T (K), P (bars) => d(entropy)/d(Moles of components) (J/K)  
➜ Moles of components, T (K), P (bars) => d^2(entropy)/d(Moles of components)^2 (J/K)  

In [ ]:
print ('Entropy = {0:12.2f} J/K'.format(Feldspar.entropy(t, p, mol=moles_end)))
print ("")
dsdm = Feldspar.entropy(t, p, mol=moles_end, deriv={'dmol':1})[0]
d2sdm2 = Feldspar.entropy(t, p, mol=moles_end, deriv={'dmol':2})[0]
for i in range (0, Feldspar.props['endmember_num']):
    print ('ds/dm[{0:>2d}] = {1:13.6e}     d2sdm2[{0:>2d}][*]:  '.format(i, dsdm[i], i), end=' ')
    for j in range (0, Feldspar.props['endmember_num']):
        print ('{0:13.6e}'.format(d2sdm2[i, j]), end=' ')
    print()

## Heat capacity and its derivatives:
➜ Moles of components, T (K), P (bars) => isobaric heat capacity (J/K)  
➜ Moles of components, T (K), P (bars) => d(isobaric heat capacity)/dT (J/K^2)  
➜ Moles of components, T (K), P (bars) => d(isobaric heat capacity)/d(Moles of components) (J/K)  

In [ ]:
print ('{0:<10s}{1:13.6f} {2:<15s}'.format('Cp', Feldspar.heat_capacity(t, p, mol=moles_end), 'J/K'))
print ('{0:<10s}{1:13.6f} {2:<15s}'.format('dCp/dT', Feldspar.heat_capacity(t, p, mol=moles_end, deriv={'dT':1}), 'J/K^2'))
print ("")
dcpdm = Feldspar.heat_capacity(t, p, mol=moles_end, deriv={'dmol':1})[0]
for i in range (0, Feldspar.props['endmember_num']):
    print ('dCp/dm[{0:>2d}]    = {1:13.6e} J/K-m'.format(i, dcpdm[i]))

## Volume and its derivatives:
➜ Moles of components, T (K), P (bars) => volume (J/bar)  
➜ Moles of components, T (K), P (bars) => d(volume)/d(Moles of components) (J/bar)  
➜ Moles of components, T (K), P (bars) => d^2(volume)/d(Moles of components)^2 (J/bar)  
➜ Moles of components, T (K), P (bars) => d(volume)/dT (J/bar-K)  
➜ Moles of components, T (K), P (bars) => d(volume)/dP (J/bar^2)  
➜ Moles of components, T (K), P (bars) => d2(volume)/dT^2 (J/bar-K^2)  
➜ Moles of components, T (K), P (bars) => d2(volume)/dTdP (J/bar^2-K)  
➜ Moles of components, T (K), P (bars) => d2(volume)/dP^2 (J/bar^3)  
➜ Moles of components, T (K), P (bars) => d2(volume)/d(Moles of components)dT (J/bar-K)  
➜ Moles of components, T (K), P (bars) => d2(volume)/d(Moles of components)dP (J/bar^2)  

In [ ]:
print ('{0:<10s}{1:13.6f} {2:<15s}'.format('Volume', Feldspar.volume(t, p, mol=moles_end), 'J/bar'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('dvdt', Feldspar.volume(t, p, mol=moles_end, deriv={'dT':1}), 'J/bar-K'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('dvdp', Feldspar.volume(t, p, mol=moles_end, deriv={'dP':1}), 'J/bar^2'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('d2vdt2', Feldspar.volume(t, p, mol=moles_end, deriv={'dT':2}), 'J/bar-K^2'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('d2vdtdp', Feldspar.volume(t, p, mol=moles_end, deriv={'dT':1,'dP':1}),  'J/bar^2-K'))
print ('{0:<10s}{1:13.6e} {2:<15s}'.format('d2vdp2', Feldspar.volume(t, p, mol=moles_end, deriv={'dP':2}), 'J/bar^3'))
print ("")
dvdm = Feldspar.volume(t, p, mol=moles_end, deriv={'dmol':1})[0]
d2vdm2 = Feldspar.volume(t, p, mol=moles_end, deriv={'dmol':2})[0]
for i in range (0, Feldspar.props['endmember_num']):
    print ('dv/dm[{0:>2d}] = {1:13.6e} J/bar-m     d2vdm2[{0:>2d}][*]:  '.format(i, dvdm[i], i), end=' ')
    for j in range (0, Feldspar.props['endmember_num']):
        print ('{0:13.6e}'.format(d2vdm2[i, j]), end=' ')
    print('J/bar-m^2')

## Accessing properties of solution species:
➜ Moles of components, T (K), P (bars) => formulae as an NSString object  
➜ Retrieves the name of the solution species at the specified index  
➜ Moles of solution species => moles of endmember components  
➜ Retrieves an elemental stoichiometry vector for the species at the specified index  
➜ Moles of components, T (K), P (bars) => chemical potentials of solution species (J)  

In [ ]:
import numpy as np
print ('Composition as a formula = ', Feldspar.compute_formula(t, p, moles_end))
mSpecies = np.zeros(Feldspar.props['species_num'])
for i in range (0, Feldspar.props['species_num']):
    print ('Species = {0:<15.15s}  elms:'.format(Feldspar.props['species_name'][i]), end=' ')
    elm = Feldspar.props['species_elms'][i]
    for j in range (0, 107):
        if elm[j] > 0.0:
            print ('[{0:>2.2s}] {1:5.2f}'.format(core.chem.PERIODIC_ORDER[j], elm[j]), end=' ')
    print ('   chemical potential = {0:12.2f} J/m'.format(Feldspar.chem_potential(t, p, mol=moles_end, endmember=i, species=True)))
    mSpecies[i] = float(i+1)
for i in range (0, Feldspar.props['species_num']):
    print ('moles of species   [{0:>2d}] = {1:5.2f}'.format(i, mSpecies[i]))
mSpToComp = Feldspar.convert_species_to_comp(mSpecies)
for i in range (0, Feldspar.props['endmember_num']):
    print ('moles of component [{0:>2d}] = {1:5.2f}'.format(i, mSpToComp[i]))

## Solution model parameters
#### Accessing, modifying, functional derivatives

In [ ]:
if Feldspar.param_props['supports_calib']:
    print ('This phase supports the Calibration protocol', end=' ')
    np = Feldspar.param_props['param_num']
    print ('and there are', np, 'parameters')
    for i in range (0, np):
        name = Feldspar.param_names[i]
        value = Feldspar.get_param_values(param_names=[name])[0]
        units = Feldspar.param_units(param_names=[name])[0]
        dgdw = Feldspar.gibbs_energy(t, p, mol=moles_end, deriv_param=[name])
        print ('Parameter {0:<10.10s} = {1:13.6e} {2:<10.10s} dggw = {3:13.6e}'.format(
            name, value, units, Feldspar.gibbs_energy(t, p, mol=moles_end, deriv_param=[name])))
        dmudw = Feldspar.gibbs_energy(t, p, mol=moles_end, deriv={'dmol':1}, deriv_param=[name])[0]
        print ('   dmu[*]dw:', end=' ')
        for j in range (0, Feldspar.props['endmember_num']):
            print ('{0:13.6e}'.format(dmudw[j]), end=' ')
        print ()
    Feldspar.set_param_values(param_names=['whabor','whorab'], param_values=['1.0', '2.0'])
    print ()
    print('Reset values of whabor and whorab: ', Feldspar.get_param_values(param_names=['whabor','whorab']))
else:
    print ('This phase does not implement the parameter calibration protocol')